In [83]:
import requests


def gdrive_link(file_id):
    return 'https://drive.google.com/uc?export=download&id={}'.format(file_id)


def parse(string):
    string = string.decode("utf-8")
    string = string.strip("'")
    string = string.split("'	'")
    return string

#get column headers for counts matrix data
#[labels for all 122 classes for manual annotation; 122 columns, 1 row] column headers for counts matrix data
file_id = '0B8AIZXhts0nWQ3c1TmRqUXRDMlE'
link = gdrive_link(file_id)
r = requests.get(link)
col_headers = r.content
col_headers = parse(col_headers)


#get labels for ciliate classes
# [subset of labels for 27 ciliate classes; 27 columns, 1 row] [note: maximum richness 27 for the plot]
file_id = '0B8AIZXhts0nWbmxUWFRtaUdRSXc'
link = gdrive_link(file_id)
r = requests.get(link)
cil_labels = r.content
cil_labels = parse(cil_labels)

#get rows labeled by date for counts matrix data
#[07-Jun-2006 to 22-Feb-2017; 1 column, 543 rows] 
file_id = '0B8AIZXhts0nWUFV2UlR3VGl0cjg'
link = gdrive_link(file_id)
r = requests.get(link)
dates = r.text
dates = dates.split("\r")
for i in range(len(dates)):
    dates[i] = dates[i].replace("'","")

#get counts matrix data
#count per class per date for all 122 classes; 122 columns, 543 rows];
file_id = '0B8AIZXhts0nWNzBIcnRvTFZzTVk'
link = gdrive_link(file_id)
r = requests.get(link)
counts_data = r.text
counts_matrix = []
count = 0
temp_mat = []
num = ''
for x in counts_data:
    if x != '\t' and x != '\r':
        num += x
    if x == '\t' or x == '\r':
        temp_mat.append(num)
        num = ''
        count +=1 
    if count == 122:
        counts_matrix.append(temp_mat)
        count = 0
        temp_mat = []
counts_matrix.append(temp_mat)

#this is a 2nd matrix of ml_analyzed per class per date for all 122 classes; 
#122 columns, should be 543 rows - remove last row of NaNs
#if a class has an NaN for a particular date then it could not be adequately observed / exclude that class]
file_id = '0B8AIZXhts0nWOXhjRnZ2YWtWa1U'
link = gdrive_link(file_id)
r = requests.get(link)
ml_analyzed = r.text
ml_matrix = []
count = 0
temp_mat = []
num = ''
for x in ml_analyzed:
    if x != '\t' and x != '\r' and x != '\n':
        num += x
    if x == '\t' or x == '\r':
        temp_mat.append(num)
        num = ''
        count +=1 
    if count == 122:
        ml_matrix.append(temp_mat)
        count = 0
        temp_mat = []
del ml_matrix[-1]

for x in range(len(ml_matrix)):
    for i in range(len(ml_matrix[x])):
        if ml_matrix[x][i] != 'NaN':
            ml_matrix[x][i] = float(ml_matrix[x][i]) 

In [88]:
counts_mat = pd.DataFrame.from_records(counts_matrix, columns=col_headers)
counts_mat.index = dates
counts_mat.index = df.index.to_datetime()
print(df)

           Asterionellopsis Ceratium Chaetoceros Corethron Cylindrotheca  \
2006-06-07                0        0          78         0           142   
2006-06-14                0        2           0         0             0   
2006-06-20                3        2         357         0            71   
2006-06-21                2        2         231         0            19   
2006-07-09                0        0           0         3             0   
2006-07-10                2        0         276         1            21   
2006-07-14                0        2           0         5             0   
2006-07-17               16        0         390         2            18   
2006-07-24                5        6         116         2            14   
2006-07-27                0        0          33         0            11   
2006-07-29                0        2           0         8             0   
2006-08-25                2        5         336        24            22   
2006-08-26  

/Users/sydruzicka/desktop/ENTER/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: to_datetime is deprecated. Use pd.to_datetime(...)
  This is separate from the ipykernel package so we can avoid doing imports until
